In [3]:
import pandas as pd
import numpy as np

from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

In [4]:
df = pd.read_excel('a_b test ecom.xlsx')
df.head()

,id,time,con_treat,page,converted
0,851104,11:48.6,control,old_page,0
1,804228,01:45.2,control,old_page,0
2,661590,55:06.2,treatment,new_page,0
3,853541,28:03.1,treatment,new_page,0
4,864975,52:26.2,control,old_page,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   id         294478 non-null  int64 
 1   time       294478 non-null  object
 2   con_treat  294478 non-null  object
 3   page       294478 non-null  object
 4   converted  294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


In [6]:
df.describe()

,id,converted
count,294478.000000,294478.000000
mean,787974.124733,0.119659
std,91210.823776,0.324563
min,630000.000000,0.000000
25%,709032.250000,0.000000
50%,787933.500000,0.000000
75%,866911.750000,0.000000
max,945999.000000,1.000000


### Проверим и почистим данные

In [8]:
df[(df['con_treat'] == 'control') & (df['page'] != 'old_page')]

,id,time,con_treat,page,converted
22,767017,58:15.0,control,new_page,0
240,733976,11:16.4,control,new_page,0
490,808613,44:01.3,control,new_page,0
846,637639,09:52.7,control,new_page,1
850,793580,25:33.7,control,new_page,1
...,...,...,...,...,...
293894,741581,49:03.4,control,new_page,0
293996,942612,52:28.2,control,new_page,0
294200,928506,32:10.5,control,new_page,0
294253,886135,49:20.5,control,new_page,0


In [9]:
df[(df['con_treat'] == 'treatment') & (df['page'] != 'new_page')]

,id,time,con_treat,page,converted
308,857184,34:59.8,treatment,old_page,0
327,686623,26:40.7,treatment,old_page,0
357,856078,29:30.4,treatment,old_page,0
685,666385,11:54.8,treatment,old_page,0
713,748761,47:44.4,treatment,old_page,0
...,...,...,...,...,...
293773,688144,34:50.5,treatment,old_page,1
293817,876037,15:09.0,treatment,old_page,1
293917,738357,37:55.7,treatment,old_page,0
294014,813406,25:33.2,treatment,old_page,0


In [10]:
df.id.nunique()

290584

In [11]:
df_clean = df[((df['con_treat'] == 'control') & (df['page'] == 'old_page'))|((df['con_treat'] == 'treatment') & (df['page'] == 'new_page'))]
df_clean.shape

(290585, 5)

In [12]:
df_clean.con_treat.value_counts()

con_treat
treatment    145311
control      145274
Name: count, dtype: int64

In [13]:
n_c = df_clean[df_clean['con_treat'] == 'control'].shape[0]
n_t = df_clean[df_clean['con_treat'] == 'treatment'].shape[0]
conv_c = df_clean[(df_clean['con_treat'] == 'control')].converted.sum()
conv_t = df_clean[(df_clean['con_treat'] == 'treatment')].converted.sum()

print(n_c, n_t)
print(conv_c, conv_t)

145274 145311
17489 17264


In [14]:
from statsmodels.stats import proportion

#### Проверим значима ли разница хи квадратом

In [17]:
chisq, pvalue, table = proportion.proportions_chisquare(np.array([conv_c,conv_t]),
                                                        np.array([n_c,n_t]))
print(round(chisq, 3), round(pvalue, 3))

1.72 0.19


#### Проверим значима ли разница z тестом

In [20]:
z_score, z_pvalue = proportion.proportions_ztest(np.array([conv_c,conv_t]),
                                                        np.array([n_c,n_t]))
print(round(z_score, 3), round(z_pvalue, 3))

1.312 0.19


#### Рассчитаем мощность теста